Base_model 1_(DNN)

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Activation
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.optimizers import Adadelta

# Assuming the images are resized/preprocessed to 60x60 grayscale
input_shape = (224, 224, 1) # Last dimension 1 for grayscale

model = Sequential([
    # Flattening the 60x60 input image to a 3600-element vector
    Flatten(input_shape=input_shape),
    
    # Example of adding a few hidden layers with batch normalization and ReLU activation
    Dense(512, kernel_initializer=HeNormal()), # He initialization
    BatchNormalization(),
    Activation('relu'),

    Dense(256, kernel_initializer=HeNormal()),
    BatchNormalization(),
    Activation('relu'),
    
    # Output layer for binary classification
    Dense(1, activation='sigmoid')
])

# Compilation of the model using Adadelta optimizer and binary crossentropy loss
# since this is a binary classification problem
model.compile(optimizer=Adadelta(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Model summary to check the architecture
model.summary()

# Example training call, replace x_train, y_train, x_val, y_val with your data
# model.fit(x_train, y_train, epochs=5000, validation_data=(x_val, y_val), callbacks=[early_stopping])


ModuleNotFoundError: No module named 'tensorflow'

##Data Preprocessing

In [ ]:
import os
import cv2
import numpy as np

# Assuming imagePaths is defined and filled with your image paths as before
data = []
labels = []

for imagePath in imagePaths:
    # Extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]
    # Load the image
    image = cv2.imread(imagePath)
    # Convert it to grayscale
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Resize it to be a fixed 60x60 pixels, ignoring aspect ratio
    image = cv2.resize(image, (60, 60))
    # Add a channel dimension:
    image = np.expand_dims(image, axis=-1)
    # Update the data and labels lists, respectively
    data.append(image)
    labels.append(label)

# Convert the data and labels to NumPy arrays while scaling the pixel intensities to the range [0, 1]
data = np.array(data) / 255.0
labels = np.array(labels)


Basemodel_2 (DNN)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Model parameters
num_filters = 32  # Starting with more filters since the image is larger
kernel_size = (3, 3)  # Smaller kernel size
dropout_rate = 0.5
learning_rate = 0.001

# Input shape is larger now
input_shape = (224, 224, 1)  # Adjust the last dimension if your images are RGB

model = Sequential([
    Conv2D(num_filters, kernel_size, activation='relu', input_shape=input_shape, padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(num_filters * 2, kernel_size, activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(num_filters * 4, kernel_size, activation='relu', padding='same'),
    # Removed one pooling layer to prevent too much downsampling
    
    Conv2D(num_filters * 8, kernel_size, activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),  # Final pooling layer
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(dropout_rate),
    Dense(2, activation='softmax')  # Adjust based on your number of classes
])

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Basemodel 3 (DCNN)

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Add, concatenate
from tensorflow.keras.optimizers import Adam

def create_cnn_stream(input_shape):
    """
    Create a CNN stream with skip connections.
    """
    inputs = Input(shape=input_shape)
    
    # Conv Block 1
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    skip_1 = x  # Skip connection
    
    # Conv Block 2
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    skip_2 = x  # Skip connection
    
    # Conv Block 3 - With skip connection from Block 1
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = Add()([x, skip_1])  # Adding skip connection
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    # Conv Block 4 - With skip connection from Block 2
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = Add()([x, skip_2])  # Adding skip connection
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    return Model(inputs=inputs, outputs=x)

# Assuming grayscale images
input_shape = (224, 224, 1)

# Create two streams
stream1_model = create_cnn_stream(input_shape)
stream2_model = create_cnn_stream(input_shape)

# Assuming you're merging features from both streams
combined = concatenate([stream1_model.output, stream2_model.output])

# Flatten and add dense layers for classification
x = Flatten()(combined)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)  # Binary classification output

# Final model
model = Model(inputs=[stream1_model.input, stream2_model.input], outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


ModuleNotFoundError: No module named 'tensorflow'